# KeT5 를 활용하여 Translation 구현하기 (Large PEFT Version) - small data

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 한국어 T5모델 중 하나인 KeT5-Large 모델을 이용하여 English-Korean 번역 문제를 풀어보겠습니다.
이번 실습에서는 다양한 PEFT 테크닉을 이용해서 큰 모델을 작은 GPU에 적용하는 방법을 사용해보겠습니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(훈련 소요 시간 : 1시간, colab 기준)

In [1]:
# Performance Metric
!pip install -q sacrebleu


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from IPython.display import Markdown, display

In [3]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Dataset 준비

In [4]:
## It takes time! (about 10min)
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# the following code is sometimes not working at AWS.
#dataset = load_dataset("bongsoo/news_talk_en_ko", column_names=["english", "korean"])

ori_dataset = load_dataset("bongsoo/news_talk_en_ko")
def correct_dataform(from_data):
    from_data.set_format(type="pandas")
    df = from_data["train"][:]
    example_0 = list(df.columns)
    example_0_df = pd.DataFrame({col: [value] for col, value in zip(('english', 'korean'), example_0)})

    df.columns = ('english', 'korean')
    en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
    en_ko_df.head()
    dataset = Dataset.from_pandas(en_ko_df)
    return DatasetDict({
                        "train": dataset
                    })
dataset = correct_dataform(ori_dataset)

### Dataset 살펴보기

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1300000
    })
})

In [6]:
dataset['train']['english'][:10]

["Skinner's reward is mostly eye-watering.",
 'Even some problems can be predicted.',
 'Only God will exactly know why.',
 "Businesses should not overlook China's dispute.",
 'Slow-beating songs often float over time.',
 "I can't even consider uninsured treatments.",
 'Including Jesus, thirteen people eat together.',
 'Uncertified companies should use government-created platforms.',
 "At least someone won't be missed.",
 "Children's evaluations show that they work."]

In [7]:
dataset['train']['korean'][:10]

['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.',
 '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.',
 '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.',
 '중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.',
 '박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.',
 '보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.',
 '예수까지 합치면 모두 열세 명이 함께 식사를 하는 것이다.',
 '인증을 받지 못한 기업은 정부가 만든 플랫폼을 활용해야 한다.',
 '적어도 누군가 보고 싶은 일이 일어나진 않을 듯 합니다.',
 '아이들 평가를 해보면 효과가 있다는 것을 알 수 있다.']

### Train / Valid / Test 분할하기

In [8]:
from datasets import DatasetDict
# 데이터셋을 훈련, 검증, 테스트 세트로 나누기
train_testvalid = dataset["train"].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# 최종적으로 train, valid, test 세트 생성
train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']

# DatasetDict로 조합
datasets = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1040000
    })
    validation: Dataset({
        features: ['english', 'korean'],
        num_rows: 130000
    })
    test: Dataset({
        features: ['english', 'korean'],
        num_rows: 130000
    })
})

적절히 잘 나눠졌음을 확인할 수 있습니다.

### Data 전처리 하기

### Model 설정하기

In [10]:
MODEL_ID = "KETI-AIR/ke-t5-large-ko"

In [11]:
import transformers
from transformers import AutoTokenizer    
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# fast version 인지 아닌지 확인
#assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
max_token_length = 64

In [13]:
#print(tokenizer.vocab)

보통은 Encoder - Decoder 모델에 있어서, Target sequence output을 만들 때 아래처럼, tokenizer를 target langauge에 맞게 바꿔줄 필요가 있습니다. 
> tokenizer.src_lang = 'enr-XX'

> tokenizer.tgt_lang = 'kr-KR'

하지만 우리가 사용하는 모델은 같은 Vocab file을 사용하기 때문에 굳이 해당 루틴을 적용할 필요는 없습니다.

In [14]:
e = dataset["train"]['english'][0]
k = dataset["train"]['korean'][0]

e_output = tokenizer(e)
with tokenizer.as_target_tokenizer():
    k_output = tokenizer(k)

print( e_output['input_ids']  )
print( k_output['input_ids'] )
print("-"*50)
print( tokenizer.decode( e_output['input_ids'] ))
print( tokenizer.decode( k_output['input_ids'] ))

[388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1]
[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1]
--------------------------------------------------
Skinner's reward is mostly eye-watering.</s>
스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.</s>


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


> 위 결과에서 보듯이, 이번에 사용하는 ""KETI-AIR/ke-t5-base" 모델은 Encoder, Decoder 모두 같은 vocabulary 를 사용하고 있는것을 알 수 있습니다.

### Prefix 넣어주기
T5 original paper에서는 모든 Task 에 task prefix를 넣도록 하는 방법을 제안합니다. 즉, 영어에서 한국어로 번역한다면 "Translate English to Korean : "이라는 prefix term 을 입력의 가장 앞에 넣어주는 방식입니다. 

그 이후에 나온 T5의 구현체마다 미미하게 다른 방식을 씁니다. 만약 Downstream task가 '번역'만 다룬다면 굳이 필요 없을 수도 있습니다. 

본 실습에서는 Prefix도 넣어주는 방법으로 구현해봅니다.

In [15]:
max_token_length = 64

ENABLE_PREFIX = True

source_lang = "English"
target_lang = "Korean"

if ENABLE_PREFIX:
    prefix = f"translate {source_lang} to {target_lang}: "
else:
    prefix = ""
    
source_lang = source_lang.lower()
target_lang = target_lang.lower()
    
def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    
    tokenized_examples = tokenizer(
                            inputs,
                            text_target=targets, 
                            max_length=max_token_length, 
                            truncation=True
                         )
    return tokenized_examples

### tokenizer test해보기

In [16]:
print(preprocess_function(dataset['train'][:2]).keys())
out = preprocess_function(dataset['train'][:2])
print(out)
print("-"*50)
print("Input Tokens  : ", out['input_ids'][0])
print("Input IDs     : ", tokenizer.decode(out['input_ids'][0]) )
print("-"*50)
print("Output Tokens : ", out['labels'][0])
print("Output IDs    : ", tokenizer.decode(out['labels'][0]) )

KeysView({'input_ids': [[35017, 6373, 10, 10698, 103, 388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1], [35017, 6373, 10, 10698, 103, 4014, 322, 3170, 147, 67, 23274, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1], [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]]})
{'input_ids': [[35017, 6373, 10, 10698, 103, 388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1], [35017, 6373, 10, 10698, 103, 4014, 322, 3170, 147, 67, 23274, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1], [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]]}
--------------------------------------------------
Input Tokens  :

### Dataset 전체에 걸쳐 전처리하기

In [17]:
tokenized_datasets = datasets.map(
                        preprocess_function, 
                        batched=True,
                        remove_columns=dataset["train"].column_names,
                     )

Map:   0%|          | 0/1040000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1040000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 130000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 130000
    })
})

In [19]:
# TRUE -> 모든 데이터 사용
# FALSE -> 테스트를 위한 소규모 데이터만 사용
#USE_ALL_DATA = False
USE_ALL_DATA = True
    
if USE_ALL_DATA: 
    #train_dataset = tokenized_datasets["train"]
    #valid_dataset = tokenized_datasets["validation"]
    #eval_dataset = tokenized_datasets["test"]
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(20000))
    valid_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(6000)) # smaller
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(6000)) # smaller
else:    
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    valid_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))    

> 전체 데이터를 다 쓰게 되면, Colab 15G기준 10시간안에 실험이 안끝날 수 있습니다.
> half data를 써서 진행합니다.

## Model 준비 하기

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

tokenizer_id = MODEL_ID

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=True,
)

# 모델 로딩
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16,
    use_safetensors=True,  
)

### Model Inference 수행해보기

In [22]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

번역에 맞는 몇 가지 입출력 전처리를 추가합니다.
1. Batch 내 입력, 출력의 최대 길이 조정
2. Input뿐만 아니라 Output레이블에도 Padding을 추가해줍니다.
3. Decoder input 에 start symbol에 해당하는 pad를 넣어줍니다.

In [23]:
sample_batch = [tokenized_datasets["train"][i] for i in range(1, 3)]

In [24]:
print( sample_batch[0] )
print( sample_batch[1] )

{'input_ids': [35017, 6373, 10, 10698, 103, 825, 612, 4, 38, 18786, 42417, 106, 22917, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [712, 34099, 70, 4311, 15, 392, 224, 4067, 242, 12154, 605, 3, 1]}
{'input_ids': [35017, 6373, 10, 10698, 103, 283, 17, 8, 6957, 13, 23266, 43, 8351, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [188, 3913, 10098, 31990, 3707, 47523, 1487, 3, 1]}


In [25]:
batch = data_collator(sample_batch)

In [26]:
batch.keys()

KeysView({'input_ids': tensor([[35017,  6373,    10, 10698,   103,   825,   612,     4,    38, 18786,
         42417,   106, 22917,     3,     1],
        [35017,  6373,    10, 10698,   103,   283,    17,     8,  6957,    13,
         23266,    43,  8351,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  712, 34099,    70,  4311,    15,   392,   224,  4067,   242, 12154,
           605,     3,     1],
        [  188,  3913, 10098, 31990,  3707, 47523,  1487,     3,     1,  -100,
          -100,  -100,  -100]]), 'decoder_input_ids': tensor([[    0,   712, 34099,    70,  4311,    15,   392,   224,  4067,   242,
         12154,   605,     3],
        [    0,   188,  3913, 10098, 31990,  3707, 47523,  1487,     3,     1,
             0,     0,     0]])})

In [27]:
print( batch['input_ids'] )

tensor([[35017,  6373,    10, 10698,   103,   825,   612,     4,    38, 18786,
         42417,   106, 22917,     3,     1],
        [35017,  6373,    10, 10698,   103,   283,    17,     8,  6957,    13,
         23266,    43,  8351,     3,     1]])


In [28]:
print( batch['attention_mask'] )

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [29]:
print( batch['labels'] )

tensor([[  712, 34099,    70,  4311,    15,   392,   224,  4067,   242, 12154,
           605,     3,     1],
        [  188,  3913, 10098, 31990,  3707, 47523,  1487,     3,     1,  -100,
          -100,  -100,  -100]])


In [30]:
print( batch['decoder_input_ids'] )

tensor([[    0,   712, 34099,    70,  4311,    15,   392,   224,  4067,   242,
         12154,   605,     3],
        [    0,   188,  3913, 10098, 31990,  3707, 47523,  1487,     3,     1,
             0,     0,     0]])


collator로 인해 2가지가 새로 변한 것을 알 수 있습니다.
- decoder_input_ids는 앞에 0 즉, pad symbol 이 추가되었습니다.
- labels 와 decoder_input_ids 를 살펴보면 하나씩 오른쪽으로 shift되었음을 확인할 수 있습니다.

In [32]:
batch = {k: v.to('cuda:0') for k, v in batch.items()}
outputs = model(**batch)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [33]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [34]:
print(outputs.loss)
print(outputs.logits)

tensor(16.1562, device='cuda:0', dtype=torch.float16,
       grad_fn=<NllLossBackward0>)
tensor([[[-4.9961, -0.4934,  2.0762,  ..., -0.8926,  0.3369,  1.2539],
         [-1.3643, -0.4211, -0.0734,  ..., -0.0961,  0.2695,  0.2327],
         [-1.4189,  1.0947,  0.7603,  ..., -0.2476,  0.1538,  0.3027],
         ...,
         [-1.9160,  0.2097,  1.6436,  ..., -0.1815,  0.2913,  0.3481],
         [-3.3652,  1.2305,  1.9609,  ..., -0.3254,  0.2478,  0.3875],
         [-2.6055,  0.3989,  0.6797,  ..., -0.1113,  0.0832,  0.3054]],

        [[-5.0352, -0.5098,  2.0781,  ..., -0.8975,  0.3408,  1.2588],
         [ 1.8584,  1.1260, -0.1193,  ..., -0.0334,  0.0892, -0.0054],
         [-1.8105,  0.0316,  0.1410,  ..., -0.0598,  0.2474,  0.3198],
         ...,
         [-5.2656, -0.5635,  2.1367,  ..., -0.9199,  0.3513,  1.2988],
         [-5.1914, -0.5488,  2.1113,  ..., -0.9131,  0.3474,  1.2881],
         [-5.1680, -0.5415,  2.1055,  ..., -0.9111,  0.3459,  1.2842]]],
       device='cuda:0', dty

### 예측 결과를 Symbol 로 바꾸기

In [35]:
import torch
best_output = torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() 
tokenizer.convert_ids_to_tokens( best_output )

['▁Gill', 'iq', '딧', 'ai', '딧', '딧', '▁그지없', 'ai', '딧', '딧', '딧', '딧', '딧']

어떻게든 답은 나오지만, 제대로 된 결과가 아님을 알 수 있습니다. 훈련을 통해 제대로 된 결과를 얻어보죠.

## Fine-Tuning 수행하기

### Gradient Checkpointing 적용하기

In [36]:
# Gradient Checkpointing 활성화
# model.gradient_checkpointing_enable()

### LoRA 적용하기

In [37]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)

### Evaluation Metric 

In [38]:
import evaluate
metric = evaluate.load("sacrebleu")

### SacreBLEU
SacreBLEU는 자연어 처리(NLP) 분야에서 기계 번역의 품질을 평가하기 위해 널리 사용되는 도구입니다. BLEU(BiLingual Evaluation Understudy) 점수는 기계 번역의 출력과 사람이 번역한 참조 번역 사이의 유사도를 측정하는 방법 중 하나로, SacreBLEU는 이 BLEU 점수를 보다 일관되고 신뢰할 수 있는 방식으로 계산합니다.

In [39]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result


위의 compute_metrics 는 [Transformer 번역 예제](https://huggingface.co/course/chapter7/4?fw=pt#metrics)에서 가져왔습니다.

### Huggingface 의 Trainer 활용하여 훈련하기

In [40]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    f"{MODEL_ID}-lora-finetuned-english-to-korean",
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=7,             # 훈련 에폭 수
    #num_train_epochs=10,             # 훈련 에폭 수
    #num_train_epochs=10,            # 훈련 에폭 수
    #max_steps=100,

    per_device_train_batch_size=60,  # 디바이스 당 훈련 배치 크기 (to be 20)
    auto_find_batch_size=True, 
    
    ## ---- Learning Rate ----- ##
    warmup_steps=200,               # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율
    learning_rate=1e-3,              # 데이터 셋 마다 적정값을 찾아야 합니다.
    
    ## - Logging & Checkpoint - ##
    save_strategy="epoch",           # 에폭마다 모델을 저장
    logging_steps=10,                # 로그를 기록할 스텝 간격
    
    ## ---- Evaluations ----- ##
    predict_with_generate=True,
    eval_strategy="epoch",
)


- fp16=False is default value

### Trainer 설정하기

In [41]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    
    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=valid_dataset,   # < test
    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_12121/4111572105.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [42]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [43]:
#torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./results")

In [ ]:
# finish a 'run' session
wandb.finish()

## 훈련된 모델 Inference

In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [ ]:
input_text = ['translate English to Korean: '+x for x in datasets['validation']['english'][:2] ]

In [ ]:
#input_text = datasets['validation']['english'][:2] 

In [ ]:
inputs = tokenizer(input_text, 
                   return_tensors="pt", 
                   padding=True, 
                   max_length=max_token_length)

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

In [ ]:
tokenizer.decode(inputs['input_ids'][1])

In [ ]:
machines = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
    max_new_tokens=25,
)

machines.shape

In [ ]:
print( 
        [
            input_text[0], 
            input_text[1]
        ]            
    )

In [ ]:
print( [ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(m)) for m in machines
])